# PandasDataFrameOutputParser

**Pandas DataFrame**은 Python 프로그래밍 언어에서 널리 사용되는 데이터 구조로, 데이터 조작 및 분석을 위한 강력한 도구입니다. DataFrame은 구조화된 데이터를 효과적으로 다루기 위한 포괄적인 도구 세트를 제공하며, 이를 통해 데이터 정제, 변환 및 분석과 같은 다양한 작업을 수행할 수 있습니다.

이 **출력 파서**는 사용자가 임의의 Pandas DataFrame을 지정하여 해당 DataFrame에서 데이터를 추출하고, 이를 형식화된 사전(dictionary) 형태로 조회할 수 있게 해주는 LLM(Large Language Model) 기반 도구입니다.


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


In [3]:
import pprint
from typing import Any, Dict

import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [4]:
# ChatOpenAI 모델 초기화 (gpt-3.5-turbo 모델 사용을 권장합니다)
model = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

`format_parser_output` 함수는 파서 출력을 사전 형식으로 변환하고 출력 형식을 지정하는 데 사용됩니다. 

In [5]:
# 출력 목적으로만 사용됩니다.
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    # 파서 출력의 키들을 순회합니다.
    for key in parser_output.keys():
        # 각 키의 값을 딕셔너리로 변환합니다.
        parser_output[key] = parser_output[key].to_dict()
    # 예쁘게 출력합니다.
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

- `titanic.csv` 데이터를 읽어온 뒤 DataFrame 을 로드하여 `df` 변수에 할당합니다.
- PandasDataFrameOutputParser를 사용하여 DataFrame을 파싱합니다.


In [6]:
# 원하는 Pandas DataFrame을 정의합니다.
df = pd.read_csv("./data/storageinfo.csv")
df.head()

,category,name,storageDays,storageDesc,storageMethod
0,기타,두부,7,5~7일,물에 담가 밀폐용기에 넣어 냉장 보관. 매일 물을 갈아주면 더 오래 보관 가능합니다.
1,야채,콩나물,5,3~5일,씻지 않은 상태로 물기를 제거하고 밀폐용기에 담아 냉장 보관하세요. 빛을 차단하면 ...
2,야채,숙주나물,4,2~4일,"콩나물보다 쉽게 무르므로, 씻지 않고 밀폐용기에 담아 냉장 보관 후 가급적 빨리 소..."
3,야채,고구마,30,2~4주,10~15°C의 서늘하고 어두운 곳에 신문지로 감싸 보관하세요. (냉장 보관 시 맛...
4,야채,감자,30,2~4주,빛이 없는 서늘하고 통풍이 잘 되는 곳에 보관하세요. 사과와 함께 두면 싹 나는 것...


In [10]:
df.tail()

,category,name,storageDays,storageDesc,storageMethod
55,과일,레몬,28,3~4주,낱개로 랩이나 비닐 팩에 싸서 냉장 보관하면 수분 증발을 막을 수 있습니다.
56,과일,망고,5,후숙 후 3~5일,검은 반점이 생길 때까지 실온에서 후숙시킨 후 냉장 보관하세요.
57,과일,체리,7,5~7일,씻지 않은 상태로 밀폐용기에 담아 냉장 보관하세요.
58,과일,석류,21,2~3주,통째로 서늘한 실온이나 냉장 보관하세요.
59,과일,아보카도,5,후숙 후 3~5일,"단단하면 실온에서 후숙시키고, 익은 후에는 냉장 보관하세요."


In [7]:
df.info

<bound method DataFrame.info of    category    name  storageDays        storageDesc  \
0        기타      두부            7               5~7일   
1        야채     콩나물            5               3~5일   
2        야채    숙주나물            4               2~4일   
3        야채     고구마           30               2~4주   
4        야채      감자           30               2~4주   
5        야채      양파           60              1~2개월   
6        야채      마늘           60              1~2개월   
7        야채       파           14               1~2주   
8        야채      생강           21               2~3주   
9        야채      오이            7               5~7일   
10       야채      가지            7               5~7일   
11       야채      호박            7               5~7일   
12       야채     옥수수            3               2~3일   
13       야채      상추            7               5~7일   
14       야채      깻잎            7               5~7일   
15       야채     쌈채소            5               3~5일   
16       야채      고추           14 

In [8]:
# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = PandasDataFrameOutputParser(dataframe=df)

# 파서의 지시사항을 출력합니다.
print(parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

컬럼에 대한 값을 조회하는 예제입니다.

In [9]:
# 열 작업 예시입니다.
df_query = "Age column 을 조회해 주세요."


# 프롬프트 템플릿을 설정합니다.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{question}\n",
    input_variables=["question"],  # 입력 변수 설정
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },  # 부분 변수 설정
)

# 체인 생성
chain = prompt | model | parser

# 체인 실행
parser_output = chain.invoke({"question": df_query})

# 출력
format_parser_output(parser_output)

OutputParserException: Invalid column: Age. The possible columns are category, name, storageDays, storageDesc, storageMethod.. Please check the format instructions.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

첫 번째 행을 검색하는 예시입니다.

In [11]:
# 행 조회 예시입니다.
df_query = "Retrieve the first row."

# 체인 실행
parser_output = chain.invoke({"question": df_query})

# 결과 출력
format_parser_output(parser_output)

{'0': {'category': '기타',
       'name': '두부',
       'storageDays': 7,
       'storageDesc': '5~7일',
       'storageMethod': '물에 '
                        '담가 '
                        '밀폐용기에 '
                        '넣어 '
                        '냉장 '
                        '보관. '
                        '매일 '
                        '물을 '
                        '갈아주면 '
                        '더 '
                        '오래 '
                        '보관 '
                        '가능합니다.'}}


특정 열에서 일부 행의 평균을 검색하는 작업 예제입니다.

In [ ]:
# row 0 ~ 4의 평균 나이를 구합니다.
df["Age"].head().mean()

In [ ]:
# 임의의 Pandas DataFrame 작업 예시, 행의 수를 제한합니다.
df_query = "Retrieve the average of the Ages from row 0 to 4."

# 체인 실행
parser_output = chain.invoke({"question": df_query})

# 결과 출력
print(parser_output)

다음은 요금(Fare) 에 대한 평균 가격을 산정하는 예시입니다.

In [ ]:
df.head()

In [ ]:
# 잘못 형식화된 쿼리의 예시입니다.
df_query = "Calculate average `Fare` rate."

# 체인 실행
parser_output = chain.invoke({"question": df_query})

# 결과 출력
print(parser_output)

In [ ]:
# 결과 검증
df["Fare"].mean()